In [46]:
import numpy as np
import torch 
import torchvision
from torchvision import transforms
from torch import nn
from torch.nn.parameter import Parameter
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, TensorDataset
import random
import copy
import matplotlib.pyplot as plt
import math
import pandas as pd

In [102]:
class SelfAttention(nn.Module):
    def __init__(self, embedding_dim, qkv_dim):
        super().__init__()
        self.embedding_dim = embedding_dim
        self.qkv_dim = qkv_dim
        
        self.query_matrix = Parameter(torch.rand(embedding_dim, qkv_dim), requires_grad=True)
        self.key_matrix = Parameter(torch.rand(embedding_dim, qkv_dim), requires_grad=True)
        self.value_matrix = Parameter(torch.rand(embedding_dim, qkv_dim), requires_grad=True)
        
    def forward(self, embeddings):
        queries = torch.matmul(embeddings, self.query_matrix)
        keys = torch.matmul(embeddings, self.key_matrix)
        values = torch.matmul(embeddings, self.value_matrix)

        attention_scores = torch.matmul(queries, keys.T)
        scaled_attention_scores = attention_scores/math.sqrt(self.qkv_dim)
        softmaxed_attention_scores = torch.softmax(scaled_attention_scores, dim=1)
        #print(softmaxed_attention_scores, values)
        return torch.matmul(softmaxed_attention_scores, values)
        

In [125]:
class MultiheadAttention(nn.Module):
    def __init__(self, embedding_dim, qkv_dim, out_dim, n_heads=8):
        super().__init__()
        self.embedding_dim = embedding_dim
        self.qkv_dim = qkv_dim
        self.n_heads = n_heads
        
        self.attention_heads = [SelfAttention(embedding_dim, qkv_dim) for i in range(n_heads)]
        self.after_concat_multiplier_matrix = Parameter(torch.rand(qkv_dim*n_heads, out_dim), requires_grad=True)
        
    def forward(self, embeddings):
        indiv_att_outs = [attention_head(embeddings) for attention_head in self.attention_heads]
        concat_outs = torch.concat(indiv_att_outs,  dim=1)
        recombined = torch.matmul(concat_outs, self.after_concat_multiplier_matrix)
        
        return recombined

In [ ]:
def FFlayer(nn.Module)::
    def __init__(self, attention_output_dim, out_dim):
        pass
        'todo'
    
    def forward(self, attention_outs):
        pass
        'todo'

In [126]:
model2 = MultiheadAttention(5,4,3)

In [127]:
model2(embeddings)

tensor([[ 92.4192,  99.4669,  88.9275],
        [ 93.7663, 100.2391,  89.8655],
        [ 94.1293, 100.9355,  90.3312],
        [ 94.1076, 100.9584,  90.3220]], grad_fn=<MmBackward0>)

In [43]:
class TransformerEncoder(nn.Module):
    def __init__(self, embedding_dim, qkv_dim):
        super().__init__()
        
        
        